# Calculate distance and age for G150 using HI velocities

In [1]:
%matplotlib inline
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pyJam.fermi.SNR_MC.distNsep import *
import pyJam.kdistMaster as kd
from astropy import units as u
from astropy import constants as c
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
mpl.style.use('ggplot')
from pyJam.utilities.sortNumStrings import atoi, natural_keys
from uncertainties import ufloat
import uncertainties
from uncertainties.umath import *

%load_ext autoreload
%autoreload 2

home = os.environ['HOME']
desk = os.path.join(home,'Desktop')
#os.chdir(os.path.join(home,'Dropbox/CurrentWork/G150/'))

## HI, radio, and other properies

In [2]:
#For Galactic rotation curve, only negative velocities are allowed (why?), so left out 2.9 km/s
#Vlsr HI velocties[km/s], all with widths 5 km/s or less, so there is no evidence of line broadening from the SNR shock
HIvels = [-44.7, -35.9, -6.9]# left out 2.9] 

#Radio coords of the SNR
lrad = 150.3
brad = 4.5

#For rotation curve
#kidst defaults are r0=8.4e3,v0=2.54e2 from Reid et al., 2009
#Reid et al 2014 gives r0=8.34, v0=240
R0 = 8.34  # [kpc] distance to center of Galaxy
V0 = 240 # [km/s] LSR velocity 

# 1 GeV-1 TeV fit disk propertie
R_gev = 1.40 # My best fit > 1 GeV #1.46 was from ajay's poster #pointlike uniform disk fit radius [deg], +/- 0.03
lgev = 150.35
bgev = 4.551
#Distance Jack listed on the symposium poster
jacksDists= [5.5,4,0.6] #kpc
jacksSizes = np.array([288, 209,31]) #pc

## Calculate distance
### Use the fit GeV coordinates

In [3]:
dist = np.vectorize(kd.kdist) #in kpc
dnear = dist(l = lgev, b = bgev, vin = HIvels, near = True,r0 = R0,v0=V0)
dfar= dist(l = lgev, b = bgev, vin = HIvels, near = False,r0 = R0,v0=V0)
print 'HI velocites are %s km/s' %HIvels
print 'Near distances %s kpc' %dnear
print 'Far distances %s kpc \n' %dfar


HI velocites are [-44.7, -35.9, -6.9] km/s
Near distances [ 5.55732525  3.94174137  0.38461266] kpc
Far distances [ 5.55732525  3.94174137  0.38461266] kpc 



## Is it possible to use the uncertainties module to propogate uncertainties in kdist?
## I think it doesn't work because 

In [5]:
#wrapped_dist = uncertainties.wrap(dist) #can I vectorize this as well?
wrapped_dist = uncertainties.wrap(kd.kdist)

lgev_unc = ufloat(lgev, 0.03)
bgev_unc = ufloat(bgev, 0.03)
HIvels_unc = ufloat(HIvels[-1],5) #this was just the width jack mentioned, idk if this will mess things up?
R0_unc = ufloat(R0, 0.03)
V0_unc = ufloat(V0, 0.03)
dist(l = lgev_unc, b = bgev_unc, vin = HIvels_unc, near = False,r0 = R0_unc,v0=V0_unc)



## Projected diameter
### My numbers are a little different than Jack's. I don't think mine are wrong. If I do it in astropy I get my results

In [9]:
size = np.vectorize(physSep) #diam
sizes = size(t=R_gev*2, d=dnear) #t is for theta = angular radius
rads = sizes/2.
jsizes = size(t=R_gev*2, d=jacksDists) #t is for theta = angular radius, 2* for diameter/size
jrads = jsizes/2
print 'Physical diameter of source in pc: %s' %sizes
print "Physical diameter of source (using Jack's dists) in pc : %s" %jsizes
print "Physical sizes from fermi symp poster in pc: %s" %jacksSizes

Physical diameter of source in pc: [ 271.58214487  192.62982233   18.79572025]
Physical diameter of source (using Jack's dists) in pc : [ 268.78070481  195.47687622   29.32153143]
Physical sizes from fermi symp poster in pc: [288 209  31]


In [7]:
#Perseus cluster is 2 kpc away, HII clouds are at 4kpc
#what are the projected radii
print 'Perseus radius = %s pc, HII dist. radius = %s pc' %(size(t=R_gev*2, d=2)/2,  size(t=R_gev*2, d=4)/2)

Perseus radius = 48.8692190558 pc, HII dist. radius = 97.7384381117 pc


In [8]:
#Just double checking I get the same dist for give ang size and phys radius
dFromSizes(18.79572025,R_gev*2)

0.38461265829770591

## For SNRs in the LMC, SMC and M33 there is a sharp cutoff in the SNR size distribution above D~60pc, which implies a distance of 1.3 kpc. Use this below

## Our closest distances are pretty off and it will make a big difference in the age!

In [10]:
dmaxJ = 1.3 #kpc, this is the number Jack quoted
smax = 60 #pc
maxSize = physSep(t=R_gev*2, d=1.3)
print 'Using 1 GeV fit radius and d = %s kpc, size = %s' %(dmaxJ,maxSize)

dmax = dFromSizes(smax,R_gev*2)
print 'Max distance assuming 60pc diameter and > 1 GeV fit radius (%s deg): %s kpc' %(R_gev,dmax)

#double check my dist calc in astropy
#assume dmax, find sep btwn G150 center crds and something at the edge of the fit radius
#can I get back 60pc?
g150 = SkyCoord(l=150.3*u.degree, b=4.5*u.degree, frame='galactic',distance = dmax*u.kpc)
g150_1_4deg = SkyCoord(l=150.3*u.degree, b=5.9*u.degree, frame='galactic',distance = dmax*u.kpc)
asepMax = g150.separation(g150_1_4deg).deg
psepMax = 2*(g150.separation_3d(g150_1_4deg).to(u.pc))

print 'Assuming d=%s kpc, and sources seperated by %s deg, diam =  %s' %(dmax,asepMax,psepMax)

Using 1 GeV fit radius and d = 1.3 kpc, size = 63.5299847726
Max distance assuming 60pc diameter and > 1 GeV fit radius (1.4 deg): 1.22776670385 kpc
Assuming d=1.22776670385 kpc, and sources seperated by 1.4 deg, diam =  59.9985073858 pc


## Use 1.22 as max distance assuming SNR sizes cutoff at 60pc

# Sedov-Taylor phase stuff (see section 6 Truelove & McKee 1999)
Radius of the expanding shell

\begin{equation} R_{ST} = 0.26 (n_H/cm^{-3})^{-1/5}(t_{ST}/yr)^{2/5} (E_{ST}/4\times 10^{50} erg)^{1/5} pc = 0.31(E_{51}/n_0)^{1/5}t_{y}^{2/5} pc\end{equation}

In terms of t (this is missing a factor in front)
\begin{equation}t_y = (n_0/E_{51})^{1/2}(R_{ST}/0.31)^{5/2}\end{equation} 

Velocity of expanding shell
\begin{equation}v_s = 1.23\times 10^5(E_{51}/n_0)^{1/5}t_{y}^{2/5} km s^{-1}\end{equation}

Temperature of expanding shell (don't need this right now)
\begin{equation}T_s = 2.09\times 10^{11} = (E^{51}/n_0)^{2/5}t^{-6/5} K\end{equation}

In [11]:
#make this all work with astropy units?
def Rst(t,n0=1,E=1e51):
    """Sedov-Taylor phase SNR radius as 
       function of time. Is this 
       Chevalier 1982 or Truelove & McKee 1999? 
       I think the former? See Reynolds 2008
       
       n0 = ambient density [ cm^-3]
       t = SNR age [yr]
       E = SN explosion energy erg
       """
    return 0.314*t**(2/5.)*((E/1.e51)/n0)**(1/5.)

def Tst(R,n0=1, E=1e51):
    """Sedov-Taylor phase SNR 
        age as function of radius
       
       n0 = ambient density [ cm^-3]
       R = Sedov-Taylor phase SNR radius [pc]. 
       E = SN explosion energy erg
    """
    return (n0/(E/1e51))**(1/2.)*(R/(0.314))**(5/2.)


In [31]:
# def Ru(t,n0=1,E=1e51):
#     """test other version to see if I'm missing mu: umd"""
    
#     return 12.4*((E/1e51)/n0)**(1/5.)*(t/1e4)**(2/5.) #[pc]

# def Rj(t,n0=1,E=1e51):
#     """test other version to see if I'm missing mu:jack"""
    
#     return (2.025*((E/1e51)*t**2.)/n0)**(1/5.) #[pc]

In [32]:
# ru = Ru(t = 1e4, n0 = 2, E = 2e51)
# rj = Rj(t = 1e4*np.pi*1e7, n0 = 2, E = 2e51)
# print 'Ru = %s, Rj = %s' %(ru,rj)

Ru = 12.4, Rj = 45724.0777528


## Test Rst and Tst on data for W44 to make sure it's right

In [12]:
#This matches Uchiyama 2012 so long as the prefactor is 0.314, bit off if it's just 0.31
r_w44 = Rst(t = 1e4, n0 = 2., E = 2e51)
t_w44 = Tst(R = 12.5,n0 = 2., E = 2e51)
print 'Test W44: R = %s, T = %s' %(r_w44,t_w44)


Test W44: R = 12.5005651554, T = 9998.86977867


## This is all from using n = 1/cm^3

In [13]:
n = 1.
T = np.vectorize(Tst)
R = np.vectorize(Rst)
T_g150 = T(R = rads,n0 = n, E=1e51)
jT_g150 = T(R = jrads,n0 = n, E=1e51)
jpostT_g150 = T(R = jacksSizes/2.,n0 = n, E=1e51)
maxD_g150_T = T(R = smax/2.,n0 = n, E=1e51)
print 'For n0 = %s cm^-3, E_0 = 10^51 erg \n\nWith physical radii in pc %s\nAges in kyr:%s\n' %(n,rads,T_g150/1.e3)
print "With radii calc'd from Jack's dists in pc %s\nAges in kyr: %s\n" %(jrads,jT_g150/1.e3)
print "Using radii from poster in pc %s\nAges in kyr: %s\n" %(jacksSizes/2.0,jpostT_g150/1.e3)
print "Assuming max size of 60 pc (d=1.23kpc), \nAges in kyr: %s\n" %(maxD_g150_T/1.e3)

For n0 = 1.0 cm^-3, E_0 = 10^51 erg 

With physical radii in pc [ 135.79107243   96.31491116    9.39786013]
Ages in kyr:[ 3889.1431202   1647.81973863     4.90058947]

With radii calc'd from Jack's dists in pc [ 134.3903524    97.73843811   14.66076572]
Ages in kyr: [ 3789.62390902  1709.38294399    14.89592627]

Using radii from poster in pc [ 144.   104.5   15.5]
Ages in kyr: [ 4503.831251    2020.53606159    17.1200538 ]

Assuming max size of 60 pc (d=1.23kpc), 
Ages in kyr: 89.223450932



# Redo the age calc using the upper density limit Dan derived

## Using physical sizes calucalted above, estimate age if G150 is in ST phase
### SN explosion energy = $\mathrm{10^{51}}$ erg

### ROSAT  thermal X-ray UL on  emitting density: nEmit = 0.02 (d/1kpc)^(-0.5) cm^-3 

In [15]:
# n scales with distance 
# should I vectorize n as well?

def nEmit(d):
    """Max X-ray emitting density 
       d: dist in kpc
    """
    return 0.02*d**(-0.5)


In [19]:
# Get d assuming n
def dEmit(n):
    return (n/0.02)**-2

In [24]:
dEmit(0.1)

0.04

In [16]:
#using just the best and max distance
nEbest = nEmit(dnear[-1])*u.cm**(-3)
nEmax = nEmit(dmax)*u.cm**(-3)
nHmax = nEmax/4.
nHbest = nEbest/4.
print "Max nH from maxD:  nEmit = %0.5f, nH = nEmit/4 = %0.5f, for d = %0.5f" %(nEmax.value,nHmax.value,dmax)
print "Max nH from bestD: nEmit = %0.5f, nH = nEmit/4 = %0.5f, for d = %0.5f" %(nEbest.value,nHbest.value,dnear[-1])

Max nH from maxD:  nEmit = 0.01805, nH = nEmit/4 = 0.00451, for d = 1.22777
Max nH from bestD: nEmit = 0.03225, nH = nEmit/4 = 0.00806, for d = 0.38461


# For nHbest

In [17]:
#for nHmax, Rst = smax/2., d = dmax
T = np.vectorize(Tst)
Tmax = T(R = smax/2.,n0 = nHmax.value, E=1e51)

#for nHbmax Rounding!, Rst = smax/2., d = dmax
nHmaxR = np.round(nHmax.value,3)
TmaxR = T(R = smax/2,n0 = nHmaxR, E=1e51)


#for nHbest, Rst = rads[-1], d = dnear[-1]
Tbest = T(R = rads[-1],n0 = nHbest.value, E=1e51)

#for nHbest Rounding!, Rst = rads[-1], d = dnear[-1]
nHbestR = np.round(nHbest.value,2)
TbestR = T(R = rads[-1],n0 = nHbestR, E=1e51)

#if density was typical of younger rems, n = 0.1
Tp1 = T(R = rads[-1],n0 = 0.1, E=1e51)
Tp1_max = T(R = smax/2,n0 = 0.1, E=1e51)

print "For max d =1.22kpc (s=60pc),   nH = %0.5f Ages in kyr: %0.5f\n" %(nHmax.value,Tmax/1.e3)
print "For max d =1.22kpc (s=60pc),   Rounded nH = %0.5f Ages in kyr: %0.5f\n" %(nHmaxR,TmaxR/1.e3)
print "For best d =0.38kpc (s=19pc),  nH = %0.5f Ages in kyr: %0.5f\n" %(nHbest.value,Tbest/1.e3)
print "For best d =0.38kpc (s=19pc),  Rounded nH = %0.5f Ages in kyr: %0.5f\n" %(nHbestR,TbestR/1.e3)
print "For typical young SNR density at d = 0.38kpc, nH = %0.5f Ages in kyr: %0.5f\n" %(0.1,Tp1/1.e3)
print "For typical young SNR density at d = 1.22kpc, nH = %0.5f Ages in kyr: %0.5f\n" %(0.1,Tp1_max/1.e3)

#and if I mix best nHbest with Tmax
TmaxMix = T(R = smax/2,n0 = nHbest.value, E=1e51)
print "For best d =1.22kpc (s=60pc),  nH = %0.5f Ages in kyr: %0.5f\n" %(nHbest.value,TmaxMix/1.e3)

For max d =1.22kpc (s=60pc),   nH = 0.00451 Ages in kyr: 5.99356

For max d =1.22kpc (s=60pc),   Rounded nH = 0.00500 Ages in kyr: 6.30905

For best d =0.38kpc (s=19pc),  nH = 0.00806 Ages in kyr: 0.44003

For best d =0.38kpc (s=19pc),  Rounded nH = 0.01000 Ages in kyr: 0.49006

For typical young SNR density at d = 0.38kpc, nH = 0.10000 Ages in kyr: 1.54970

For typical young SNR density at d = 1.22kpc, nH = 0.10000 Ages in kyr: 28.21493

For best d =1.22kpc (s=60pc),  nH = 0.00806 Ages in kyr: 8.01139



#  Since the X-ray density estimate implies a young Sedove age, I should also calculate the age of ST onset (when the SNR left the free expansion phase

## Free Expansion Phase (after explosion, prior to ST)
http://astro.berkeley.edu/~ay216/05/NOTES/Lecture11.pdf,  From that ISM book, ISM notes too,  Slane et al 2010, Truelove & McKee 1999


ST phase starts when swept up mass = ejecta mass, $R \propto t$
\begin{equation}M_{su} = 4\pi /3 R^3 \rho = 0.145n_0R^3_{pc} M_{\odot}\end{equation}
\begin{equation} R_{ST_0} = (3M_{ej}/4\pi \rho)^{1/3} = 1.9(M_{ej}/n_0)^{1/3} pc \end{equation}

ISM book says typical SNIa $M_{ej}$ =0.25 M_sol, v = 2e4 km/s, but it also says the ambient density is n = 1 cm^-3. 

In [145]:
(1/0.145)**(1/3.)

1.9034610713106543

In [148]:
def t_on(E,n,u):
    """Age in years of ST onset, when the swept up mass == ejecta mass
       E: SN energy in 1e51 erg
       n: ambient density in cm^-3
       u: shock velocity before ST phase in 1e9 cm/s = 1e4km/s
       from Cardillo thesis
    """
    Esn = E/(1e51)
    v = u/(1e9)
    return 200*(Esn/(n*v**5.))**(1/3.)

def R_on(M = 1, n=1):
    f = (1/0.145)**(1/3.)
    return f*(M/n)**(1/3.)*u.pc

def t_s(M,n,v,mu = 1.4):
    M = M.to(u.kg)
    n = n.to(u.cm**(-3))
    v = v.to(u.cm/u.s)
    dom = 4*np.pi*v**3*n*mu*c.m_p
    return (3*M/dom)**(1/3.)

#def rshock(E,):

In [150]:
#Check on W44
#it's a bit off, paper has 129
rw44 = R_on(2,2).to(u.cm)
vw44 = 1e4*(u.km/u.s)
tw44 = rw44/vw44
#print t.to(u.yr)
print rw44.to(u.pc), vw44, tw44.to(u.yr)

1.90346107131 pc 10000.0 km / s 186.118942979 yr


In [139]:
Etst =2e51*u.erg
ntst = 1*u.cm**(-3)
vtst = 1.e4*u.km/u.s#1e9*u.cm/u.s
Mtst=1*u.Msun
ttst = t_s(Mtst,ntst,vtst)
print ttst.to(u.yr)


186.170175476 yr


In [100]:
E =2e51*u.erg
n = 2*u.cm**(-3)
v = 1e9*u.cm/u.s
t_on(E.value,n.value,v.value)
#print v.to(u.km/u.s)

200.0

In [143]:
#for small nh
tG150 = t_s(Mtst,nHbest,3*vtst)
#vG150 = 1e4*(u.km/u.s)

print  tG150.to(u.yr)

309.48251267 yr


In [ ]:
#what I really want is to calculate vs

## Assuming typical PSR velocity, about how far offset would a ballastically traveling pulsar be from the center of SNR given estimated age (mine) from above?
### Gaensler & Slane  2006: v_psr ~400-500 km/s, but some can have an extreme value of v_psr = 1000 km/s. 


In [33]:
# T = np.vectorize(Tst)
# T_g150 = T(R = rads,n0 = 1, E=1e51)
# angSize
# size = np.vectorize(physSep)
# jsizes = size(t=R_gev*2, d=jacksDists)
J0426_d = physSep(0.84,dnear[-1])*u.pc
print 'Projected distance of J0426 from the center is %s' %J0426pc
print 'Angular sep of J0426 from a pulsar travelling 1000km/s is %s deg' %angSize(0.1,dnear[-1])

J0426_v = J0426pc.to(u.km)/((T_g150*u.yr).to(u.s))
print 'The velocity of J0426 if it was traveling ballistically from the center of the SNR is\n %s' %J0426_v

Projected distance of J0426 from the center is 5.63871607642 pc
Angular sep of J0426 from a pulsar travelling 1000km/s is 0.014897008266 deg
The velocity of J0426 if it was traveling ballistically from the center of the SNR is
 [    1.41766259     3.34593196  1125.06725122] km / s


In [10]:
#psrVs = np.array([400.,500.,1000.])*u.km/u.s
psrVs = np.array([1.e3])*u.km/u.s
psrDs = psrVs*(T_g150*u.yr)
psrDs = psrDs.to(u.pc)
angD = np.vectorize(angSize)
angDs = angD(s=np.asarray(psrDs),d=dnear)
print "For v_psr = 1000km/s"
print "Projected pulsar distances: %s" %psrDs.to(u.pc)
print "Pulsar angular separation: %s" %angDs
print "For ages %s kyr\n" %(T_g150/1e3)

psrVs = np.array([400.])*u.km/u.s
psrDs = psrVs*(T_g150*u.yr)
psrDs = psrDs.to(u.pc)
angDs = angD(s=np.asarray(psrDs),d=dnear)
print "For v_psr = 400km/s"
print "Projected pulsar distances: %s" %psrDs.to(u.pc)
print "Pulsar angular separation: %s" %angDs
print "For ages %s kyr" %(T_g150/1e3)


For v_psr = 1000km/s
Projected pulsar distances: [ 3977.47398066  1685.24529251     5.01189246] pc
Pulsar angular separation: [ 41.00758226  24.49613856   0.74662203]
For ages [ 3889.1431202   1647.81973863     4.90058947] kyr

For v_psr = 400km/s
Projected pulsar distances: [ 1590.98959227   674.098117       2.00475699] pc
Pulsar angular separation: [ 16.4030329    9.79845543   0.29864881]
For ages [ 3889.1431202   1647.81973863     4.90058947] kyr


## 3FGL J0426 is ~0.84 deg from the center of the SNR. If the PSR was travelling at 1000 km/s, It would be ~0.1 deg away from how far out J0426 is. So it's no impossible that J0426 is the pulsar associated with G150 I guess.